In [1]:
! pip install --upgrade google-cloud-bigquery-storage

In [1]:
import google, pathlib, shutil, numpy as np, pandas as pd, geopandas as gpd
from google.cloud import aiplatform, bigquery
from shapely.ops import orient
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

proj_id = 'cmat-315920'
root_path = '/home/jupyter'
root_path = pathlib.Path(root_path)
raw_data_path = root_path / 'raw_data'
cred, proj = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
bqclient = bigquery.Client(credentials = cred, project = proj)

def lower_cols(df):
    def f(x):
        if x[-2:].isnumeric():
            x = x[:-2]
        return x.lower()
    return df.rename(columns={x:f(x) for x in df.columns})

def get_states():
    qry = f"""
    select
        state_fips_code as fips
        , state_postal_abbreviation as abbr
        , state_name as name
    from
        bigquery-public-data.census_utility.fips_codes_states
    where
        state_fips_code <= '56'
    """
    return bqclient.query(qry).result().to_dataframe()

def get_data(name, yr, state, url, overwrite=False, silent=True):
    print(f"Get {state['name']} {yr} {name}", end='...' )
    path = f"{name}/{yr}/{state['abbr']}"
    file = raw_data_path / f"{path}/{path.replace('/', '_')}.parquet"
    path = file.parent
    if overwrite:
        shutil.rmtree(path, ignore_errors=True)
    path.mkdir(parents=True, exist_ok=True)
    %cd -q {path}
    data = path / 'data.zip'
    if data.is_file():
        print('already have it')
    else:
        print(f'fetching from {url}', end = '...')
        if silent:
            suf = '&> /dev/null'
        else:
            suf = ''
        ! wget -O {data} {url} {suf}
        ! unzip -o {data} {suf}
    #     ! rm {file}
        print('success')

    try:
        df = gpd.read_parquet(file)
    except:
        try:
            df = pd.read_parquet(file)
        except:
            df = None
    return file, df

def write_data(df, file):
    df.to_parquet(file)
    
#     table_id = f"{proj_id}.geo.hi"
#     job = bqclient.load_table_from_dataframe(df, table_id)  # Make an API request.
#     job.result()

    table_id = f"{proj_id}.{file.stem.split('_')[0]}.{file.stem}"#cmat-315920.BLOCKGROUP_CENTROIDS.blockgroup_centroids_0{i}"
    print(table_id)
    job = bqclient.load_table_from_dataframe(df, table_id)  # Make an API request.
    job.result()  # Wait for the job to complete.

states = get_states()
states = states.iloc[[39,43]]
overwrite = False
# overwrite = True

In [2]:
# Get block assignment files

def get_assignments(yr, state, overwrite=False, silent=True):
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'

    name = 'assignments'
    url = f"https://www2.census.gov/geo/docs/maps-data/data/baf"
    if yr == 2020:
        url += '2020'
    url += f"/BlockAssign_ST{state['fips']}_{state['abbr']}.zip"
    file, df = get_data(name, yr, state, url, overwrite, silent)
    if df is None:
        L = []
        for f in file.parent.iterdir():
            col = f.name.lower().split('_')[-1][:-4]
            if col == 'aiannh' or f.suffix != '.txt':
                continue
            try:
                if yr == 2020:
                    sep = '|'
                else:
                    sep = ','
                assign = lower_cols(pd.read_csv(f, sep=sep, dtype=str))
                if col == 'vtd':
                    assign['district'] = assign['countyfp'] + assign['district'].str[-4:]
                    assign.drop(columns='countyfp', inplace=True)
                assign.columns = ['geoid', col]
                L.append(assign.set_index('geoid'))
            except:
                pass
        df= pd.concat(L, axis=1).reset_index()
        write_data(df, file)
    return df
    
for i, state in states.iterrows():
    for yr in [2020, 2010]:
        assignments = get_assignments(yr, state, overwrite)
        display(assignments.head(2))

Get Rhode Island 2020 assignments...fetching from https://www2.census.gov/geo/docs/maps-data/data/baf2020/BlockAssign_ST44_RI.zip...success
cmat-315920.assignments.assignments_2020_RI


,geoid,sduni,cd,sldl,cdp,sdsec,vtd,sldu,sdelm
0,440010301001000,00030,01,066,NaN,NaN,0010101,032,NaN
1,440010301001001,00030,01,066,NaN,NaN,0010101,032,NaN


Get Rhode Island 2010 assignments...fetching from https://www2.census.gov/geo/docs/maps-data/data/baf/BlockAssign_ST44_RI.zip...success
cmat-315920.assignments.assignments_2010_RI


,geoid,sduni,cd,sldl,cdp,sdsec,vtd,sldu,sdelm
0,440010301001000,00030,01,066,NaN,NaN,NaN,009,NaN
1,440010301001001,00030,01,066,NaN,NaN,NaN,009,NaN


Get Texas 2020 assignments...fetching from https://www2.census.gov/geo/docs/maps-data/data/baf2020/BlockAssign_ST48_TX.zip...success
cmat-315920.assignments.assignments_2020_TX


,geoid,sduni,cdp,cd,sldu,sdelm,sdsec,sldl,vtd
0,480019501001000,19770,NaN,05,003,NaN,NaN,008,0010018
1,480019501001001,19770,27300,05,003,NaN,NaN,008,0010018


Get Texas 2010 assignments...fetching from https://www2.census.gov/geo/docs/maps-data/data/baf/BlockAssign_ST48_TX.zip...success
cmat-315920.assignments.assignments_2010_TX


,geoid,sduni,cdp,cd,sldu,sdelm,sdsec,sldl,vtd
0,480019501001000,19770,NaN,05,003,NaN,NaN,008,0010018
1,480019501001001,19770,27300,05,003,NaN,NaN,008,0010018


In [11]:
def get_crosswalks(yr, state, overwrite=False, silent=True):
    yrs = [2010, 2020]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'

    name = 'crosswalks'
    url = f"https://www2.census.gov/geo/docs/maps-data/data/rel2020/t10t20/TAB2010_TAB2020_ST"
#     if yr == 2020:
#          url += '2020'
    url += f"{state['fips']}.zip"
    file, df = get_data(name, yr, state, url, overwrite, silent)
    
    if df is None:
        for f in file.parent.iterdir():
            if f.suffix == '.txt':
                df = pd.read_csv(f, sep="|", dtype=str) 
        write_data(df, file)
    return df


for i, state in states.iterrows():
    for yr in [2010, 2020]:
        crosswalks = get_crosswalks(yr, state, True)
        display(crosswalks.head(2))


Get Rhode Island 2010 crosswalks...fetching from https://www2.census.gov/geo/docs/maps-data/data/rel2020/t10t20/TAB2010_TAB2020_ST44.zip...success
cmat-315920.crosswalks.crosswalks_2010_RI


,STATE_2010,COUNTY_2010,TRACT_2010,BLK_2010,BLKSF_2010,AREALAND_2010,AREAWATER_2010,BLOCK_PART_FLAG_O,STATE_2020,COUNTY_2020,TRACT_2020,BLK_2020,BLKSF_2020,AREALAND_2020,AREAWATER_2020,BLOCK_PART_FLAG_R,AREALAND_INT,AREAWATER_INT
0,09,011,705200,3000,NaN,594,1237702,p,44,009,051000,1039,NaN,134157,0,p,594,0
1,25,005,630300,2046,NaN,4307,0,p,44,007,011404,1025,NaN,129174,0,p,1110,0


Get Rhode Island 2020 crosswalks...fetching from https://www2.census.gov/geo/docs/maps-data/data/rel2020/t10t20/TAB2010_TAB2020_ST44.zip...success
cmat-315920.crosswalks.crosswalks_2020_RI


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/_pandas_helpers.py:380: UserWarning: Pyarrow could not determine the type of columns: BLKSF_2010, BLKSF_2020.
  ", ".join(field.name for field in unknown_type_fields)


,STATE_2010,COUNTY_2010,TRACT_2010,BLK_2010,BLKSF_2010,AREALAND_2010,AREAWATER_2010,BLOCK_PART_FLAG_O,STATE_2020,COUNTY_2020,TRACT_2020,BLK_2020,BLKSF_2020,AREALAND_2020,AREAWATER_2020,BLOCK_PART_FLAG_R,AREALAND_INT,AREAWATER_INT
0,09,011,705200,3000,NaN,594,1237702,p,44,009,051000,1039,NaN,134157,0,p,594,0
1,25,005,630300,2046,NaN,4307,0,p,44,007,011404,1025,NaN,129174,0,p,1110,0


Get Texas 2010 crosswalks...fetching from https://www2.census.gov/geo/docs/maps-data/data/rel2020/t10t20/TAB2010_TAB2020_ST48.zip...success
cmat-315920.crosswalks.crosswalks_2010_TX


,STATE_2010,COUNTY_2010,TRACT_2010,BLK_2010,BLKSF_2010,AREALAND_2010,AREAWATER_2010,BLOCK_PART_FLAG_O,STATE_2020,COUNTY_2020,TRACT_2020,BLK_2020,BLKSF_2020,AREALAND_2020,AREAWATER_2020,BLOCK_PART_FLAG_R,AREALAND_INT,AREAWATER_INT
0,05,091,020900,1225,NaN,1795,769,p,48,037,011201,2022,NaN,284514,2680,p,89,13
1,05,091,021000,7000,NaN,20478895,384408,p,48,067,950101,3066,NaN,7249885,0,p,2871,0


Get Texas 2020 crosswalks...fetching from https://www2.census.gov/geo/docs/maps-data/data/rel2020/t10t20/TAB2010_TAB2020_ST48.zip...success
cmat-315920.crosswalks.crosswalks_2020_TX


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/_pandas_helpers.py:380: UserWarning: Pyarrow could not determine the type of columns: BLKSF_2010, BLKSF_2020.
  ", ".join(field.name for field in unknown_type_fields)


,STATE_2010,COUNTY_2010,TRACT_2010,BLK_2010,BLKSF_2010,AREALAND_2010,AREAWATER_2010,BLOCK_PART_FLAG_O,STATE_2020,COUNTY_2020,TRACT_2020,BLK_2020,BLKSF_2020,AREALAND_2020,AREAWATER_2020,BLOCK_PART_FLAG_R,AREALAND_INT,AREAWATER_INT
0,05,091,020900,1225,NaN,1795,769,p,48,037,011201,2022,NaN,284514,2680,p,89,13
1,05,091,021000,7000,NaN,20478895,384408,p,48,067,950101,3066,NaN,7249885,0,p,2871,0


In [ ]:
https://www2.census.gov/geo/tiger/TIGER2010BLKPOPHU/tabblock2010_44_pophu.zip
https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2010/tl_2010_44_tabblock10.zip.
def get_Block_pop(yr, state, overwrite=False, silent=True):
    yrs = [2010, 2020]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'

    name = 'crosswalks'
    url = f"https://www2.census.gov/geo/docs/maps-data/data/rel2020/t10t20/TAB2010_TAB2020_ST"
#     if yr == 2020:
#          url += '2020'
    url += f"{state['fips']}.zip"
    file, df = get_data(name, yr, state, url, overwrite, silent)
    
    if df is None:
        for f in file.parent.iterdir():
            if f.suffix == '.txt':
                df = pd.read_csv(f, sep="|", dtype=str) 
        write_data(df, file)
    return df


for i, state in states.iterrows():
    for yr in [2010, 2020]:
        crosswalks = get_crosswalks(yr, state, True)
        display(crosswalks.head(2))

    
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'

    name = 'Block_pop'
    url = f"https://www2.census.gov/geo/docs/maps-data/data/baf"
    if yr == 2020:
        url += '2020'
    url += f"/BlockAssign_ST{state['fips']}_{state['abbr']}.zip"
    file, df = get_data(name, yr, state, url, overwrite, silent)
    if df is None:
        L = []
        for f in file.parent.iterdir():
            col = f.name.lower().split('_')[-1][:-4]
            if col == 'aiannh' or f.suffix != '.txt':
                continue
            try:
                if yr == 2020:
                    sep = '|'
                else:
                    sep = ','
                assign = lower_cols(pd.read_csv(f, sep=sep, dtype=str))
                if col == 'vtd':
                    assign['district'] = assign['countyfp'] + assign['district'].str[-4:]
                    assign.drop(columns='countyfp', inplace=True)
                assign.columns = ['geoid', col]
                L.append(assign.set_index('geoid'))
            except:
                pass
        df= pd.concat(L, axis=1).reset_index()
        write_data(df, file)
    return df
    
for i, state in states.iterrows():
    for yr in [2020, 2010]:
        assignments = get_assignments(yr, state, overwrite)
        display(assignments.head(2))

In [45]:
# TX elections

def get_elections_TX(overwrite=False, silent=True):
    name = 'elections'
    yr = 2020
    state = states.query('abbr=="TX"').iloc[0]
    url = f'https://data.capitol.texas.gov/dataset/aab5e1e5-d585-4542-9ae8-1108f45fce5b/resource/253f5191-73f3-493a-9be3-9e8ba65053a2/download/{yr}-general-vtd-election-data.zip'
    file, df = get_data(name, yr, state, url, overwrite, silent)

    if df is None:
        L = []
        print(file)
        for f in file.parent.iterdir():
            if f.is_file():
                w = f.name.split('_')
                if w.pop(-1) == 'Returns.csv':
                    df = pd.read_csv(f)
                    df['yr'] = w.pop(0)
                    df['race'] = '_'.join(w)
                    L.append(df.copy())
        df = lower_cols(pd.concat(L, axis=0, ignore_index=True)).astype(str).astype({'votes':int, 'yr':int})
        df.to_parquet(file)
    return df
elections = get_elections_TX(overwrite)
display(elections.head(2))

Get Texas 2020 elections...fetching from https://data.capitol.texas.gov/dataset/aab5e1e5-d585-4542-9ae8-1108f45fce5b/resource/253f5191-73f3-493a-9be3-9e8ba65053a2/download/2020-general-vtd-election-data.zip...success
/home/jupyter/raw_data/elections/2020/TX/elections_2020_TX.parquet


,county,fips,vtd,cntyvtd,office,name,party,incumbent,votes,yr,race
0,Aransas,7,0001,70001,U.S. Rep 27,Barrera,D,N,2,2018,Congress_Dist_27_Election
1,Aransas,7,0002,70002,U.S. Rep 27,Barrera,D,N,14,2018,Congress_Dist_27_Election


In [ ]:
file.name.split('_')[0]

In [14]:
# shapefiles
def get_shapefiles(yr, state, level, overwrite=False, silent=True):
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'
    levels = ['tract', 'bg', 'tabblock']
    assert level in levels, f'level must be in {levels} - got {level}'
    name = f'shapefiles_{level}'
    
    url = f"https://www2.census.gov/geo/tiger/TIGER{yr}/{level.upper()}"
    if yr == 2010:
        url += '/2010'
    elif yr == 2020 and level == 'tabblock':
        url += '20'
    url += f"/tl_{yr}_{state['fips']}_{level}{str(yr)[-2:]}"
    if yr == 2020 and level in ['tract', 'bg']:
        url = url[:-2]
    url += '.zip'
    
    file, df = get_data(name, yr, state, url, overwrite, silent)
    if df is None:
        df = lower_cols(gpd.read_file(file.parent))[['geoid', 'aland', 'geometry']]
        df['geometry'] = df['geometry'].apply(lambda p: orient(p, -1))
        df.to_parquet(file)
    return df
    
for i, state in states.iterrows():
    for yr in [2020, 2010]:
        for level in ['tract', 'bg', 'tabblock']:
            geo = get_shapefiles(yr, state, level, overwrite)

Get Rhode Island 2020 shapefiles_tract...fetching from https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_44_tract.zip...success
Get Rhode Island 2020 shapefiles_bg...fetching from https://www2.census.gov/geo/tiger/TIGER2020/BG/tl_2020_44_bg.zip...success
Get Rhode Island 2020 shapefiles_tabblock...fetching from https://www2.census.gov/geo/tiger/TIGER2020/TABBLOCK20/tl_2020_44_tabblock20.zip...success
Get Rhode Island 2010 shapefiles_tract...fetching from https://www2.census.gov/geo/tiger/TIGER2010/TRACT/2010/tl_2010_44_tract10.zip...success
Get Rhode Island 2010 shapefiles_bg...fetching from https://www2.census.gov/geo/tiger/TIGER2010/BG/2010/tl_2010_44_bg10.zip...success
Get Rhode Island 2010 shapefiles_tabblock...fetching from https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2010/tl_2010_44_tabblock10.zip...success
Get Texas 2020 shapefiles_tract...fetching from https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_48_tract.zip...success
Get Texas 2020 shapefiles_bg..

KeyboardInterrupt: 

In [ ]:
def get_geo(state, yr, level):
    yr = str(yr)
    if level == 'blockgroup':
        a = 'bg'
    elif level == 'block':
        a = 'tabblock'
    else:
        a = level

    path = root_path / f"geo/{level}/{yr}/{state['abbr']}"
    path.mkdir(exist_ok=True, parents=True)
    try:
        geo = gpd.read_file(path)
    except:
        %cd {path}
        url = f"https://www2.census.gov/geo/tiger/TIGER{yr}/{a.upper()}"
        
        if yr == '2010':
            url += '/2010'
        elif yr == '2020' and level == 'block':
            url += '20'
        url += f"/tl_{yr}_{state['fips']}_{a}{yr[-2:]}"
        if yr == '2020' and level in ['blockgroup', 'tract']:
            url = url[:-2]
        url += '.zip'
        ! wget -O geo.zip {url}
        ! unzip geo.zip
        ! rm geo.zip
        geo = gpd.read_file(path)

    geo = lower_cols(geo)[['geoid', 'aland', 'geometry']].apply(lambda p: orient(p, -1))
    geo.columns = ['geo_id', 'geometry']

    table_id = f"{proj_id}.geo.{level}-{yr}-{state['abbr']}"
    job = bqclient.load_table_from_dataframe(geo.to_wkt(), table_id)  # Make an API request.
    job.result()
    return geo


for i, state in states.iterrows():
    print(state['name'])
    for level in ['tract', 'blockgroup', 'block']:
        print(level)
        for yr in [2010, 2020]:
            print(yr)
            geo = get_geo(state, yr, level)

In [ ]:
cols = ['FILEID', 'FILETYPE', 'STUSAB', 'CHARITER', 'SEQUENCE', 'LOGRECNO', 'SEX BY AGE% Total population%Total', 'SEX BY AGE%Total population%Total%Male', 'SEX BY AGE%Total population%Total%Male%Under 5 years', 'SEX BY AGE%Total population%Total%Male%5 to 9 years', 'SEX BY AGE%Total population%Total%Male%10 to 14 years', 'SEX BY AGE%Total population%Total%Male%15 to 17 years', 'SEX BY AGE%Total population%Total%Male%18 and 19 years', 'SEX BY AGE%Total population%Total%Male%20 years', 'SEX BY AGE%Total population%Total%Male%21 years', 'SEX BY AGE%Total population%Total%Male%22 to 24 years', 'SEX BY AGE%Total population%Total%Male%25 to 29 years', 'SEX BY AGE%Total population%Total%Male%30 to 34 years', 'SEX BY AGE%Total population%Total%Male%35 to 39 years', 'SEX BY AGE%Total population%Total%Male%40 to 44 years', 'SEX BY AGE%Total population%Total%Male%45 to 49 years', 'SEX BY AGE%Total population%Total%Male%50 to 54 years', 'SEX BY AGE%Total population%Total%Male%55 to 59 years', 'SEX BY AGE%Total population%Total%Male%60 and 61 years', 'SEX BY AGE%Total population%Total%Male%62 to 64 years', 'SEX BY AGE%Total population%Total%Male%65 and 66 years', 'SEX BY AGE%Total population%Total%Male%67 to 69 years', 'SEX BY AGE%Total population%Total%Male%70 to 74 years', 'SEX BY AGE%Total population%Total%Male%75 to 79 years', 'SEX BY AGE%Total population%Total%Male%80 to 84 years', 'SEX BY AGE%Total population%Total%Male%85 years and over', 'SEX BY AGE%Total population%Total%Female', 'SEX BY AGE%Total population%Total%Female%Under 5 years', 'SEX BY AGE%Total population%Total%Female%5 to 9 years', 'SEX BY AGE%Total population%Total%Female%10 to 14 years', 'SEX BY AGE%Total population%Total%Female%15 to 17 years', 'SEX BY AGE%Total population%Total%Female%18 and 19 years', 'SEX BY AGE%Total population%Total%Female%20 years', 'SEX BY AGE%Total population%Total%Female%21 years', 'SEX BY AGE%Total population%Total%Female%22 to 24 years', 'SEX BY AGE%Total population%Total%Female%25 to 29 years', 'SEX BY AGE%Total population%Total%Female%30 to 34 years', 'SEX BY AGE%Total population%Total%Female%35 to 39 years', 'SEX BY AGE%Total population%Total%Female%40 to 44 years', 'SEX BY AGE%Total population%Total%Female%45 to 49 years', 'SEX BY AGE%Total population%Total%Female%50 to 54 years', 'SEX BY AGE%Total population%Total%Female%55 to 59 years', 'SEX BY AGE%Total population%Total%Female%60 and 61 years', 'SEX BY AGE%Total population%Total%Female%62 to 64 years', 'SEX BY AGE%Total population%Total%Female%65 and 66 years', 'SEX BY AGE%Total population%Total%Female%67 to 69 years', 'SEX BY AGE%Total population%Total%Female%70 to 74 years', 'SEX BY AGE%Total population%Total%Female%75 to 79 years', 'SEX BY AGE%Total population%Total%Female%80 to 84 years', 'SEX BY AGE%Total population%Total%Female%85 years and over', 'SEX BY AGE (WHITE ALONE)% People who are White alone%Total', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Male', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Male%Under 5 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Male%5 to 9 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Male%10 to 14 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Male%15 to 17 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Male%18 and 19 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Male%20 to 24 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Male%25 to 29 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Male%30 to 34 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Male%35 to 44 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Male%45 to 54 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Male%55 to 64 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Male%65 to 74 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Male%75 to 84 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Male%85 years and over', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Female', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Female%Under 5 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Female%5 to 9 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Female%10 to 14 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Female%15 to 17 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Female%18 and 19 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Female%20 to 24 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Female%25 to 29 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Female%30 to 34 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Female%35 to 44 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Female%45 to 54 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Female%55 to 64 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Female%65 to 74 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Female%75 to 84 years', 'SEX BY AGE (WHITE ALONE)%People who are White alone%Total%Female%85 years and over', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)% Black or African American alone%Total', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Male', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Male%Under 5 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Male%5 to 9 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Male%10 to 14 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Male%15 to 17 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Male%18 and 19 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Male%20 to 24 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Male%25 to 29 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Male%30 to 34 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Male%35 to 44 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Male%45 to 54 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Male%55 to 64 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Male%65 to 74 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Male%75 to 84 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Male%85 years and over', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Female', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Female%Under 5 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Female%5 to 9 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Female%10 to 14 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Female%15 to 17 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Female%18 and 19 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Female%20 to 24 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Female%25 to 29 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Female%30 to 34 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Female%35 to 44 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Female%45 to 54 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Female%55 to 64 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Female%65 to 74 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Female%75 to 84 years', 'SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)%Black or African American alone%Total%Female%85 years and over', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)% People who are American Indian and Alaska Native alone%Total', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Male', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Male%Under 5 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Male%5 to 9 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Male%10 to 14 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Male%15 to 17 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Male%18 and 19 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Male%20 to 24 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Male%25 to 29 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Male%30 to 34 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Male%35 to 44 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Male%45 to 54 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Male%55 to 64 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Male%65 to 74 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Male%75 to 84 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Male%85 years and over', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Female', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Female%Under 5 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Female%5 to 9 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Female%10 to 14 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Female%15 to 17 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Female%18 and 19 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Female%20 to 24 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Female%25 to 29 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Female%30 to 34 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Female%35 to 44 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Female%45 to 54 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Female%55 to 64 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Female%65 to 74 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Female%75 to 84 years', 'SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)%People who are American Indian and Alaska Native alone%Total%Female%85 years and over', 'SEX BY AGE (ASIAN ALONE)% People who are Asian alone%Total', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Male', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Male%Under 5 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Male%5 to 9 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Male%10 to 14 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Male%15 to 17 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Male%18 and 19 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Male%20 to 24 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Male%25 to 29 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Male%30 to 34 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Male%35 to 44 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Male%45 to 54 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Male%55 to 64 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Male%65 to 74 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Male%75 to 84 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Male%85 years and over', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Female', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Female%Under 5 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Female%5 to 9 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Female%10 to 14 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Female%15 to 17 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Female%18 and 19 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Female%20 to 24 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Female%25 to 29 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Female%30 to 34 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Female%35 to 44 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Female%45 to 54 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Female%55 to 64 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Female%65 to 74 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Female%75 to 84 years', 'SEX BY AGE (ASIAN ALONE)%People who are Asian alone%Total%Female%85 years and over', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)% People who are Native Hawaiian and Other Pacific Islander alone%Total', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Male', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Male%Under 5 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Male%5 to 9 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Male%10 to 14 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Male%15 to 17 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Male%18 and 19 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Male%20 to 24 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Male%25 to 29 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Male%30 to 34 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Male%35 to 44 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Male%45 to 54 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Male%55 to 64 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Male%65 to 74 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Male%75 to 84 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Male%85 years and over', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Female', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Female%Under 5 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Female%5 to 9 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Female%10 to 14 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Female%15 to 17 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Female%18 and 19 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Female%20 to 24 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Female%25 to 29 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Female%30 to 34 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Female%35 to 44 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Female%45 to 54 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Female%55 to 64 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Female%65 to 74 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Female%75 to 84 years', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)%People who are Native Hawaiian and Other Pacific Islander alone%Total%Female%85 years and over', 'SEX BY AGE (SOME OTHER RACE ALONE)% People who are Some Other Race alone%Total', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Male', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Male%Under 5 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Male%5 to 9 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Male%10 to 14 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Male%15 to 17 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Male%18 and 19 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Male%20 to 24 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Male%25 to 29 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Male%30 to 34 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Male%35 to 44 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Male%45 to 54 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Male%55 to 64 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Male%65 to 74 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Male%75 to 84 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Male%85 years and over', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Female', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Female%Under 5 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Female%5 to 9 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Female%10 to 14 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Female%15 to 17 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Female%18 and 19 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Female%20 to 24 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Female%25 to 29 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Female%30 to 34 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Female%35 to 44 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Female%45 to 54 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Female%55 to 64 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Female%65 to 74 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Female%75 to 84 years', 'SEX BY AGE (SOME OTHER RACE ALONE)%People who are Some Other Race alone%Total%Female%85 years and over']

In [ ]:
def read_geo(state, yr, level):
    table_id = f"{proj_id}.geo.{level}-{yr}-{state['abbr']}"
    qry = f"""
        select
            *
        from
            {table_id}
        """
#     job = bqclient.load_table_from_dataframe(geo.to_wkt(), table_id)  # Make an API request.
    
#     job.result()  # Wait for the job to complete.
    bgs = bqclient.query(qry).result().to_dataframe()
    bgs['geometry'] = gpd.GeoSeries.from_wkt(bgs['geometry'])
    return bgs
#         table = bqclient.get_table(table_id)  # Make an API request.

state = states.iloc[0]
geo = get_geo(state, yr, level)
geo

In [ ]:
https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2010/tl_2010_01_tabblock10.zip
https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2010tl_2010_01_tabblock10.zip
    
    
https://www2.census.gov/geo/tiger/TIGER2010/BG/2010/tl_2010_01_bg10.zip
https://www2.census.gov/geo/tiger/TIGER2010/TRACT/2010/tl_2010_01_tract10.zip

https://www2.census.gov/geo/tiger/TIGER2020/TABBLOCK20/tl_2020_01_tabblock20.zip
https://www2.census.gov/geo/tiger/TIGER2020/BG/tl_2020_02_bg.zip
https://www2.census.gov/geo/tiger/TIGER2020/BG/tl_2020_01_bg20.zip
https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_01_tract.zip


In [ ]:
client = bigquery.Client()

for i in codes:
    if (i<10):
        table_id = f"cmat-315920.BLOCKGROUP_CENTROIDS.blockgroup_centroids_0{i}"
        df = pd.read_csv(f'https://www2.census.gov/geo/docs/reference/cenpop2010/blkgrp/CenPop2010_Mean_BG0{i}.txt')
        
        job = client.load_table_from_dataframe(df, table_id)  # Make an API request.
        job.result()  # Wait for the job to complete.
        table = client.get_table(table_id)  # Make an API request.
        print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))
        
    if (i>9):
        table_id = f"cmat-315920.BLOCKGROUP_CENTROIDS.blockgroup_centroids_{i}"
        df = pd.read_csv(f'https://www2.census.gov/geo/docs/reference/cenpop2010/blkgrp/CenPop2010_Mean_BG{i}.txt')
        job = client.load_table_from_dataframe(df, table_id)  # Make an API request.
        job.result()  # Wait for the job to complete.
        table = client.get_table(table_id)  # Make an API request.
        print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))
    
#     if i == 57:
#         j = 72
#         table_id = f"cmat-315920.BLOCK_CENTROIDS.block_centroid_{i}"
#         df = pd.read_csv(f'https://www2.census.gov/geo/docs/reference/cenpop2010/blkgrp/CenPop2010_Mean_BG{i}.txt')
#         job = client.load_table_from_dataframe(df, table_id)  # Make an API request.
#         job.result()  # Wait for the job to complete.
#         table = client.get_table(table_id)  # Make an API request.
#         print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))
        
        

In [ ]:
import google, pathlib, numpy as np, pandas as pd, geopandas as gpd
from google.cloud import aiplatform, bigquery
from shapely.ops import orient

proj_id = 'cmat-315920'
root_path = '/home/jupyter'
root_path = pathlib.Path(root_path)
cred, proj = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
bqclient = bigquery.Client(credentials = cred, project = proj)

In [ ]:
fips = 48
abbr = 'TX'
p = root_path / f'temp/{abbr}'
p.mkdir(parents=True, exist_ok=True)

url = f'https://www2.census.gov/geo/docs/maps-data/data/baf2020/BlockAssign_ST{fips}_{abbr}.zip'
%cd {p}
%ls
fn = 'baf.zip'
! wget -O {fn} {url}
! unzip {fn}
! rm {fn}